<a href="https://colab.research.google.com/github/Yulufu/MIDI-Sound-Generation-from-Air-Quality-Data/blob/main/Air_Quality_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests mido python-rtmidi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.0/765.0 kB 19.8 MB/s eta 0:00:00


In [ ]:
import requests

def fetch_air_quality_data(latitude, longitude, api_token):
    # Prepare the endpoint and parameters
    endpoint = f"https://api.waqi.info/feed/geo:{latitude};{longitude}/"
    params = {
        "token": api_token
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None  # or handle errors as appropriate

# Replace with your actual values
api_token = "42ed33d5b7c9a020591a73b08b51a8a0d299ed6d"
latitude = 40.729513
longitude = -73.997460 #NYU Steinhardt location

# Fetch and print data
air_quality_data = fetch_air_quality_data(latitude, longitude, api_token)
print(air_quality_data)


{'status': 'ok', 'data': {'aqi': 21, 'idx': 3307, 'attributions': [{'url': 'http://www.dec.ny.gov/', 'name': 'New York State Department of Environmental Conservation (NYSDEC)', 'logo': 'US-NYDEC.png'}, {'url': 'http://www.airnow.gov/', 'name': 'Air Now - US EPA'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [40.7143528, -74.0059731], 'name': 'New York, USA', 'url': 'https://aqicn.org/city/usa/newyork', 'location': ''}, 'dominentpol': 'o3', 'iaqi': {'co': {'v': 2.3}, 'h': {'v': 55}, 'no2': {'v': 4.9}, 'o3': {'v': 20.8}, 'p': {'v': 1004.4}, 'pm25': {'v': 12}, 't': {'v': 14.3}, 'w': {'v': 9.7}}, 'time': {'s': '2023-10-22 10:00:00', 'tz': '-04:00', 'v': 1697968800, 'iso': '2023-10-22T10:00:00-04:00'}, 'forecast': {'daily': {'o3': [{'avg': 15, 'day': '2023-10-20', 'max': 18, 'min': 10}, {'avg': 4, 'day': '2023-10-21', 'max': 11, 'min': 1}, {'avg': 10, 'day': '2023-10-22', 'max': 16, 'min': 6}, {'avg': 7, 'day': '2023-10-23', 'max': 15, 'min': 1

In [ ]:
def parse_air_quality_data(data):
    if data and data.get('status') == 'ok':
        # Extracting the air quality index (AQI)
        aqi = data['data']['aqi']

        # Extracting individual pollutant data
        iaqi = data['data']['iaqi']

        # You can access individual pollutants depending on availability
        pm25 = iaqi.get('pm25', {}).get('v')  # PM2.5 data
        pm10 = iaqi.get('pm10', {}).get('v')  # PM10 data
        co = iaqi.get('co', {}).get('v')  # Carbon Monoxide data
        no2 = iaqi.get('no2', {}).get('v')  # Nitrogen Dioxide data
        o3 = iaqi.get('o3', {}).get('v')  # Ozone data

        # You can add more pollutants here

        # Returning a dictionary with the parsed data
        return {
            'aqi': aqi,
            'pm25': pm25,
            'pm10': pm10,
            'co': co,
            'no2': no2,
            'o3': o3
            # Add more pollutants here
        }
    else:
        print("Error: Unable to retrieve or parse data")
        return None

# Assuming air_quality_data contains the response from the API
parsed_data = parse_air_quality_data(air_quality_data)
print(parsed_data)


{'aqi': 21, 'pm25': 12, 'pm10': None, 'co': 2.3, 'no2': 4.9, 'o3': 20.8}


In [ ]:
import mido
import time

# Function to normalize data
def normalize_data(value, min_value, max_value, min_midi, max_midi):
    return ((value - min_value) / (max_value - min_value)) * (max_midi - min_midi) + min_midi

In [ ]:
# Function to send MIDI messages
def send_midi_control_change(control, value, midi_port):
    # control is the control number (e.g., 1 for modulation wheel, 7 for volume, etc.)
    # value is the control value (0-127)
    msg = mido.Message('control_change', control=control, value=value)
    midi_port.send(msg)


In [ ]:
# Create a virtual MIDI port
with mido.open_output('MyVirtualMIDI', virtual=True) as outport:
    # Assuming air_quality_data contains the response from the API
    parsed_data = parse_air_quality_data(air_quality_data)

    # Normalize the data to MIDI range (0-127)
    # This is a simple example; you might need different ranges based on your data
    pm25_midi = int(normalize_data(parsed_data['pm25'], 0, 500, 0, 127))
    co_midi = int(normalize_data(parsed_data['co'], 0, 50, 0, 127))
    no2_midi = int(normalize_data(parsed_data['no2'], 0, 200, 0, 127))
    o3_midi = int(normalize_data(parsed_data['o3'], 0, 300, 0, 127))

    # Send MIDI messages
    send_midi_control_change(1, pm25_midi, outport)  # CC1 typically corresponds to modulation
    send_midi_control_change(2, co_midi, outport)    # CC2 can be assigned to a different parameter
    send_midi_control_change(3, no2_midi, outport)   # CC3 can be assigned to a different parameter
    send_midi_control_change(4, o3_midi, outport)    # CC4 can be assigned to a different parameter

    # Sleep for a bit to ensure messages are sent before closing the port
    time.sleep(0.1)

SystemError: ignored